In [1]:
## Goal of the Chunk: Load packages and other necessary functions

# I will import the same packages from the data processing file to ensure I have everything that I need
import sys
import os
import numpy as np
import openpyxl
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = "{:,.4f}".format # display float values to four decimal places

In [2]:
pwd

'C:\\Users\\Elizabeth Hora\\Downloads\\lane_lab_backup\\Elizabeth_Hora\\spring_thesis_2022\\toy_data'

In [4]:
## Goal of the Chunk: Specify file to be opened and load its information
### This chunk has been copied from my i_preprocessing_excel_data.ipynb file,
### with the exception of the amiga_truth and output_path_amiga variables- those have been appended to the original code


# Test inputs for now
# The user has two options:
# 1. Input the file path for the data to be transformed and supply a directory for the output file to be saved in.
# 2. Input the file path for the data to be transformed and supply an empty string.
#    The empty string means that it is assumed that the user wants to store their output data in the same input directory.
# Any incorrect file names or directories will be filtered out.
input_file_path = "C:\\Users\\Elizabeth Hora\\Downloads\\lane_lab_backup\\Elizabeth_Hora\\spring_thesis_2022\\toy_data\\processed_compiled_autosave.xlsx"
# output_dir_name = "C:\\Users\\Elizabeth Hora\\Box\\Lane_lab_docs\\Elizabeth_Hora\\Lane_Lab_GitHub\\plate_reader\\sample_data\\new_dir"
output_dir_name = ""
amiga_truth = "amiga"

# If the user supplies one argument, the argument_list should only contain one element.
if output_dir_name == "":
    argument_list = [os.path.isfile(input_file_path)]
# If the user supplies two arguments, the argument_list should have two elements in which both are checked.
else:
    argument_list = [os.path.isfile(input_file_path), os.path.isdir(output_dir_name)]

## This concept was borrowed from the Jupyter Notebook script for finding genes of interest that July and I wrote.
# If no "True" is found, this part stops the rest of the code from running.
# It then informs the user if the gene symbol was entered incorrectly and stops running the subsequent code.
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

    
Found = False

# Option 1
if len(argument_list) == 2:
    # Both the input file name is valid and the output directory exists
    if os.path.isfile(input_file_path) == True and os.path.isdir(output_dir_name) == True:
        # Splits the file name, called input_file_name, from the rest of the path
        input_dir, input_file_name = os.path.split(input_file_path)
        # places the word "processed_" before the original file name to indicate the data have been modified.
        output_path = os.path.join(output_dir_name, "processed_" + input_file_path)
        Found = True
    # The input file name is valid but the output directory is yet to exist
    elif os.path.isfile(input_file_path) == True and os.path.isdir(output_dir_name) == False:
        output_dir_check_above, output_dir_check = os.path.split(output_dir_name)
        # Taking everything above where the new directory is created and separating from the folder that will be created
        if os.path.isdir(output_dir_check_above) == True:
            input_dir, input_file_name = os.path.split(input_file_path)
            # Make a new directory and storing the file there with previously spliced information
            new_dir = os.path.join(output_dir_check_above, output_dir_check)
            os.mkdir(new_dir)
            # Saving the file in the new directory
            output_path = os.path.join(new_dir, ("processed_" + input_file_name))
            Found = True
        # Handles an incorrect path above where the new directory wants to be made
        else:
            print("The directory supplied does not have a valid path")
            pass
    # Handles incorrect input file- completely incorrect path supplied by at least the input, possibly the output too
    else:
        print("Either the input file name or the output directory name was entered incorrectly.")
# Option 2
elif len(argument_list) == 1:
    # Saving the output file in the same directory as the valid input file provided
    if os.path.isfile(input_file_path) == True:
        input_dir, input_file_name = os.path.split(input_file_path)
        output_path = os.path.join(input_dir, "processed_" + input_file_name)
        Found = True
    # Handles an invalid input
    else:
        print("Error: The input file name was entered incorrectly.")
# Too many arguments
elif len(argument_list) > 2:
    print("Error: Too many arguments given. There should be a maximum of two.")
# Not enough arguments given
else:
    print("Error: Not enough argument(s) given.")
    print("There should be at least one correct file path with the option of a new directory given.") 
    
    
# Stopping the rest of the script from running if there are invalid inputs
if Found == False:
    print("The script was stopped.")
    raise StopExecution

# Reading in the data from the required input    
data_imported = pd.read_excel(input_file_path, usecols=lambda x: 'Unnamed' not in x,)

# Removing the Blank index, the Channel, and the Cycle Nr. columns by index
data_imported = data_imported.drop(data_imported.columns[0:2], axis = 1)

# Removing the Temp. [°C] column by index
data_imported = data_imported.drop(data_imported.columns[1], axis = 1)

Found = False

# Handling saving data for AMiGA
if amiga_truth.lower() == "amiga":
    output_path_amiga = os.path.join(input_dir, "amiga_processed_" + input_file_name)
    Found = True
else:
    pass

# For potential future use
# Stopping the rest of the script from running if an AMiGA-formatted Excel sheet is not desired
if Found == False:
    print("No data in an AMiGA-acceptable format will be generated.")
    raise StopExecution

In [5]:
# Now there should only be the used wells and the Time [s] columns left
data_imported

,Time [s],A1,A2,A3,A4,A5,A6,A7,A8,A9,...,G10,G11,G12,H1,H2,H3,H4,H5,H6,H7
0,0.0000,0.0780,0.0820,0.0830,0.0820,0.0830,0.0780,0.0830,0.0830,0.0830,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.1000,0.1000,0.1000,0.0900
1,900.0200,0.0780,0.0820,0.0830,0.0820,0.0830,0.0780,0.0840,0.0830,0.0830,...,0.0500,0.0500,0.0500,0.1000,0.1000,0.1000,0.1100,0.1100,0.1100,0.0900
2,"1,800.0260",0.0780,0.0820,0.0830,0.0830,0.0840,0.0790,0.0860,0.0860,0.0860,...,0.1000,0.1000,0.1000,0.2000,0.2000,0.2000,0.1221,0.1221,0.1221,0.0900
3,"2,700.0420",0.0780,0.0830,0.0840,0.0850,0.0870,0.0810,0.0910,0.0890,0.0910,...,0.1500,0.1500,0.1500,0.3000,0.3000,0.3000,0.1370,0.1370,0.1370,0.0900
4,"3,600.0460",0.0780,0.0840,0.0850,0.0870,0.0910,0.0810,0.0970,0.0900,0.0960,...,0.2000,0.2000,0.2000,0.4000,0.4000,0.4000,0.1558,0.1558,0.1558,0.0900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,"53,100.6990",0.0780,0.5840,0.5690,0.6280,0.6210,0.0830,0.9780,1.0160,1.1430,...,0.3500,0.3500,0.3500,0.3000,0.3000,0.3000,0.3420,0.3420,0.3420,0.0900
60,"54,000.7160",0.0780,0.5830,0.5720,0.6280,0.6170,0.0830,0.9790,1.0180,1.1470,...,0.4000,0.4000,0.4000,0.2000,0.2000,0.2000,0.3712,0.3712,0.3712,0.0900
61,"54,900.7370",0.0780,0.5750,0.5680,0.6240,0.6200,0.0830,0.9800,1.0220,1.1530,...,0.4500,0.4500,0.4500,0.1000,0.1000,0.1000,0.4057,0.4057,0.4057,0.0900
62,"55,800.7430",0.0780,0.5990,0.5770,0.6220,0.6180,0.0830,0.9780,1.0200,1.1570,...,0.5000,0.5000,0.5000,0.2000,0.2000,0.2000,0.4468,0.4468,0.4468,0.0900


In [6]:
## Goal of the Chunk: Make a list of all 96 wells

# Creating a list of wells A1-H12
# Empty list to store all the well information
wells_list = []

# Rows in the plate reader are letters
letters_list = ["A", "B", "C", "D", "E", "F", "G", "H"]

# Iterating through the letters_list, making each letter have values 1 through 12 attached to it
# This is avoiding making a list of all wells by hand
for letter in letters_list:
    # Using the `range()` function to automatically go from 1 to 12
    for i in range(1, 13):
        # Appending concatenated strings to the empty wells_list
        wells_list.append(letter + str(i))

In [7]:
## Goal of the Chunk: Finding the dimensions of the column for placeholder values / data to be added to

# Finding the total length of the column, which will be used to 
# a) for existing data, add each value iteratively
# b) for unused wells, fill the values with a 0 placeholder
row_length = 0

# Calculating the row length by hand in a column, ignoring the header
for row in data_imported["Time [s]"]:
    row_length += 1

In [8]:
## Goal of the Chunk: Adding existing data or filler values to each of the 96 wells

# Creating an empty list
list_well_list = []

# Starting from A1 and ending at H12, this loop checks if there are data for a particular well
for well in wells_list:
    # resetting this list with every iteration
    well_data = []
    # Procedure for when there are data
    # If a well has data, it will exist as a column in the data_imported dataframe
    if well in data_imported:
        # Adding the name to the list first
        well_data.append(well)
        # Adding each data point iteratively to the well_data list
        for i in range(len(data_imported[well])):
            well_data.append(data_imported[well][i])
    # When the well went unused
    else:
        # Adding the name to the list first
        well_data.append(well)
        # Adding 0 iteratively to the well_data list, maintaining the dimensions
        for i in range(row_length):
            well_data.append(0)
    # adding the stored data to the first list, enabling the well_data to be wiped without losing the data gained
    list_well_list.append(well_data)
    
# Writing the results to a dataframe
wells_dataframe = pd.DataFrame(list_well_list)

In [9]:
## Goal of the Chunk: Adding Time [s] and the seconds as the header to the dataframe

# Adding Time [s] as the header
time_list = ["Time [s]"]

# Turning a column into a list
for time in data_imported["Time [s]"]:
    time_list.append(time)
    
# Preparing the list to become a transposed dataframe    
time_df = pd.DataFrame(time_list).T

# Concatenating the time values to become the header of the dataframe
complete_df = pd.concat([time_df, wells_dataframe])
# Changing the header
complete_df.columns = complete_df.iloc[0]
# Dropping the first row
complete_df = complete_df[1:]
# Checking to see the data are properly loaded
complete_df[10:20]

,Time [s],0.0,900.02,1800.026,2700.042,3600.046,4500.052,5400.056,6300.065,7200.074,...,48600.586,49500.604,50400.61,51300.628,52200.689,53100.699,54000.716,54900.737,55800.743,56700.749
10,A11,0.0780,0.0780,0.0780,0.0780,0.0780,0.0780,0.0780,0.0780,0.0780,...,0.0780,0.0780,0.0780,0.0780,0.0780,0.0780,0.0780,0.0780,0.0780,0.0780
11,A12,0.0820,0.0820,0.0830,0.0830,0.0850,0.0860,0.0870,0.0880,0.0890,...,0.5640,0.5770,0.5760,0.5880,0.5910,0.5920,0.6150,0.6010,0.6370,0.6290
12,B1,0.0830,0.0830,0.0830,0.0840,0.0850,0.0850,0.0850,0.0860,0.0860,...,0.5490,0.5450,0.5450,0.5470,0.5450,0.5610,0.5600,0.5720,0.5760,0.5740
13,B2,0.0830,0.0830,0.0840,0.0870,0.0890,0.0910,0.0920,0.0930,0.0950,...,0.6440,0.6380,0.6390,0.6370,0.6350,0.6370,0.6320,0.6360,0.6300,0.6280
14,B3,0.0830,0.0830,0.0850,0.0890,0.0930,0.0970,0.1000,0.1030,0.1050,...,0.5680,0.5660,0.5660,0.5680,0.5700,0.5780,0.5770,0.5790,0.5790,0.5820
15,B4,0.0780,0.0780,0.0790,0.0820,0.0830,0.0830,0.0840,0.0840,0.0850,...,0.0850,0.0840,0.0840,0.0850,0.0840,0.0840,0.0840,0.0840,0.0850,0.0860
16,B5,0.0830,0.0830,0.0860,0.0910,0.0970,0.1040,0.1140,0.1280,0.1480,...,1.1170,1.1100,1.1220,1.1230,1.1250,1.1330,1.1390,1.1450,1.1510,1.1440
17,B6,0.0840,0.0830,0.0860,0.0890,0.0910,0.0920,0.0940,0.0970,0.1020,...,1.1420,1.1380,1.1510,1.1520,1.1560,1.1630,1.1680,1.1750,1.1800,1.1700
18,B7,0.0830,0.0830,0.0880,0.0920,0.0970,0.1050,0.1170,0.1330,0.1520,...,1.0560,1.0490,1.0560,1.0540,1.0540,1.0580,1.0600,1.0620,1.0610,1.0580
19,B8,0.0840,0.0850,0.0930,0.1000,0.1120,0.1300,0.1550,0.1790,0.1990,...,1.0320,1.0260,1.0310,1.0310,1.0290,1.0340,1.0360,1.0370,1.0370,1.0350


In [10]:
## Goal of the Chunk: Writing the output to Excel

# Establishing the output path found in the second chunk
file_complete = output_path_amiga

# Writing a blank Excel file to avoid any potential duplication problems
blank_slate = pd.DataFrame()

# Writing over any existing data and replacing with a blank sheet
with pd.ExcelWriter(file_complete,
                    mode='w') as writer:  
    blank_slate.to_excel(writer, sheet_name='Data')

# Writing over the blank sheet with the complete_df, meaning it is now in an AMiGA-friendly format
with pd.ExcelWriter(file_complete,
                    mode='w', engine="openpyxl") as writer:  
    complete_df.to_excel(writer, sheet_name='Data', index = False)